In [10]:
import pandas as pd
import pyarrow
import fastparquet
import pyspark
import numpy as np
#from scipy.stats import grubbs
from scipy.spatial.distance import mahalanobis
from sklearn.neighbors import LocalOutlierFactor

In [11]:
df = pd.read_parquet('credit_card_data_da.parquet')
#df.to_csv('output.csv', index=False)
#df = spark.read.parquet("input.parquet")
#df.write.csv("output.csv")

In [12]:
df.columns = df.columns.str.replace(' ', '_')

In [13]:
df.head()

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
0,0,0,2016,1,3,10:48,66.48,Chip Transaction,-3345936507911876459,La Verne,...,09/2002,2008,No,0,NaN,NaN,NaN,NaN,NaN,8
1,0,0,2016,1,4,06:43,40.02,Chip Transaction,-34551508091458520,La Verne,...,09/2002,2008,No,1,0.0,0.0,66.48,66.48,1.0,8
2,0,0,2016,1,7,09:30,54.11,Chip Transaction,4055257078481058705,La Verne,...,09/2002,2008,No,2,0.0,0.0,106.50,106.50,2.0,8
3,0,0,2016,1,7,16:03,89.48,Chip Transaction,3414527459579106770,Monterey Park,...,09/2002,2008,No,2,0.0,0.0,106.50,106.50,2.0,8
4,0,0,2016,1,10,06:38,29.15,Chip Transaction,-5475680618560174533,Monterey Park,...,09/2002,2008,No,3,0.0,0.0,250.09,250.09,4.0,8


In [14]:
df.tail()

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
6877832,1999,1,2019,12,30,11:38,66.53,Chip Transaction,2910328604019547969,Milford,...,05/2017,2017,No,3376892,0.0,0.0,9607.94,9607.94,161.0,2
6877833,1999,1,2019,12,30,12:03,99.67,Chip Transaction,1641456720718814353,Merrimack,...,05/2017,2017,No,3376892,0.0,0.0,9607.94,9607.94,161.0,2
6877834,1999,1,2019,12,30,18:44,93.63,Chip Transaction,-5501892381419303127,Merrimack,...,05/2017,2017,No,3376892,0.0,0.0,9607.94,9607.94,161.0,2
6877835,1999,1,2019,12,30,20:09,67.60,Online Transaction,-6160036380778658394,ONLINE,...,05/2017,2017,No,3376892,0.0,0.0,9607.94,9607.94,161.0,2
6877836,1999,1,2019,12,31,20:26,51.71,Chip Transaction,2500998799892805156,Merrimack,...,05/2017,2017,No,3376893,0.0,0.0,9741.23,9741.23,161.0,2


In [15]:
print(df.dtypes)

User                        int64
Card                        int64
Year                        int64
Month                       int64
Day                         int64
                           ...   
rolling_fraud_count       float64
rolling_tran_count        float64
rolling_tran_volume       float64
transaction_count         float64
years_since_pin_change      int64
Length: 63, dtype: object


In [16]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [17]:
df[["rolling_fraud_count","rolling_tran_count"]].mean()

rolling_fraud_count      0.05
rolling_tran_count    2513.12
dtype: float64

In [18]:
df[["rolling_fraud_count"]].sum()

rolling_fraud_count   325218.00
dtype: float64

In [19]:
df[["Yearly_Income_-_Person", "Total_Debt", "FICO_Score","Num_Credit_Cards","Credit_Limit"]].describe()

,Yearly_Income_-_Person,Total_Debt,FICO_Score,Num_Credit_Cards,Credit_Limit
count,6877837.00,6877837.00,6877837.00,6877837.00,6877837.00
mean,46707.57,59898.51,711.49,3.57,15423.18
std,23646.99,52637.59,67.14,1.62,11986.59
min,1.00,0.00,488.00,1.00,0.00
25%,33106.00,19404.00,683.00,2.00,7800.00
50%,41401.00,53407.00,713.00,4.00,13460.00
75%,54124.00,86747.00,755.00,5.00,20765.00
max,280199.00,461854.00,850.00,9.00,141391.00


In [20]:
df.loc[df['Num_Credit_Cards'] == 9.00]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
393234,116,0,2016,1,1,13:29,1.46,Chip Transaction,-2006531472386210850,Novato,...,05/2013,2013,No,194344,NaN,NaN,NaN,NaN,NaN,3
393235,116,0,2016,1,2,16:30,0.85,Chip Transaction,6666504894937430109,Novato,...,05/2013,2013,No,194345,0.00,0.00,1.46,1.46,1.00,3
393236,116,0,2016,1,2,16:40,174.35,Chip Transaction,-4765964138602603082,Petaluma,...,05/2013,2013,No,194345,0.00,0.00,1.46,1.46,1.00,3
393237,116,0,2016,1,3,08:12,7.89,Swipe Transaction,-727612092139916043,Novato,...,05/2013,2013,No,194346,0.00,0.00,176.66,176.66,3.00,3
393238,116,0,2016,1,5,08:13,9.57,Swipe Transaction,-727612092139916043,Novato,...,05/2013,2013,No,194347,0.00,0.00,184.55,184.55,4.00,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4583564,1327,8,2019,12,9,20:08,4.25,Chip Transaction,-5475680618560174533,Tucson,...,11/2006,2010,No,2240163,0.00,0.00,369.83,369.83,8.00,9
4583565,1327,8,2019,12,25,11:32,24.34,Chip Transaction,4722913068560264812,Tucson,...,11/2006,2010,No,2240164,0.00,0.00,119.35,119.35,2.00,9
4583566,1327,8,2019,12,27,01:17,130.86,Chip Transaction,838425044734233142,Tucson,...,11/2006,2010,No,2240165,0.00,0.00,143.69,143.69,3.00,9
4583567,1327,8,2019,12,27,14:13,12.81,Chip Transaction,-3778631267350051946,Tucson,...,11/2006,2010,No,2240165,0.00,0.00,143.69,143.69,3.00,9


In [21]:
df.loc[df['Card_on_Dark_Web'] == 'No']

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
0,0,0,2016,1,3,10:48,66.48,Chip Transaction,-3345936507911876459,La Verne,...,09/2002,2008,No,0,NaN,NaN,NaN,NaN,NaN,8
1,0,0,2016,1,4,06:43,40.02,Chip Transaction,-34551508091458520,La Verne,...,09/2002,2008,No,1,0.00,0.00,66.48,66.48,1.00,8
2,0,0,2016,1,7,09:30,54.11,Chip Transaction,4055257078481058705,La Verne,...,09/2002,2008,No,2,0.00,0.00,106.50,106.50,2.00,8
3,0,0,2016,1,7,16:03,89.48,Chip Transaction,3414527459579106770,Monterey Park,...,09/2002,2008,No,2,0.00,0.00,106.50,106.50,2.00,8
4,0,0,2016,1,10,06:38,29.15,Chip Transaction,-5475680618560174533,Monterey Park,...,09/2002,2008,No,3,0.00,0.00,250.09,250.09,4.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6877832,1999,1,2019,12,30,11:38,66.53,Chip Transaction,2910328604019547969,Milford,...,05/2017,2017,No,3376892,0.00,0.00,9607.94,9607.94,161.00,2
6877833,1999,1,2019,12,30,12:03,99.67,Chip Transaction,1641456720718814353,Merrimack,...,05/2017,2017,No,3376892,0.00,0.00,9607.94,9607.94,161.00,2
6877834,1999,1,2019,12,30,18:44,93.63,Chip Transaction,-5501892381419303127,Merrimack,...,05/2017,2017,No,3376892,0.00,0.00,9607.94,9607.94,161.00,2
6877835,1999,1,2019,12,30,20:09,67.60,Online Transaction,-6160036380778658394,ONLINE,...,05/2017,2017,No,3376892,0.00,0.00,9607.94,9607.94,161.00,2


In [22]:
df.loc[df['Is_Fraud?'] == 'Yes']

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
1691,0,2,2016,3,6,11:18,81.93,Chip Transaction,-6680087784759370261,Claremont,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
1692,0,2,2016,3,6,12:10,297.86,Online Transaction,-3220758452254689706,ONLINE,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
2536,0,3,2016,2,23,10:26,244.23,Online Transaction,5763106017265139365,ONLINE,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
2537,0,3,2016,2,23,14:38,22.40,Swipe Transaction,-99077337081867836,Claremont,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
4731,1,1,2016,6,9,10:08,181.74,Online Transaction,-6761066196029841443,ONLINE,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6869755,1997,2,2016,9,4,10:45,1.25,Online Transaction,6455213054093379528,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869757,1997,2,2016,9,4,12:30,116.66,Online Transaction,822647939787511611,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869758,1997,2,2016,9,4,12:40,359.99,Online Transaction,8080934608468946743,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869759,1997,2,2016,9,4,13:45,351.51,Online Transaction,-4282466774399734331,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4


In [23]:
df.loc[(df['FICO_Score'] >= 712) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
1691,0,2,2016,3,6,11:18,81.93,Chip Transaction,-6680087784759370261,Claremont,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
1692,0,2,2016,3,6,12:10,297.86,Online Transaction,-3220758452254689706,ONLINE,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
2536,0,3,2016,2,23,10:26,244.23,Online Transaction,5763106017265139365,ONLINE,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
2537,0,3,2016,2,23,14:38,22.40,Swipe Transaction,-99077337081867836,Claremont,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
18013,3,1,2017,11,19,12:06,9.53,Chip Transaction,3017176960763408508,Rome,...,02/2007,2007,No,10016,0.00,0.00,438.10,438.10,12.00,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6869755,1997,2,2016,9,4,10:45,1.25,Online Transaction,6455213054093379528,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869757,1997,2,2016,9,4,12:30,116.66,Online Transaction,822647939787511611,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869758,1997,2,2016,9,4,12:40,359.99,Online Transaction,8080934608468946743,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869759,1997,2,2016,9,4,13:45,351.51,Online Transaction,-4282466774399734331,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4


In [24]:
df.loc[(df['FICO_Score'] < 711) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
4731,1,1,2016,6,9,10:08,181.74,Online Transaction,-6761066196029841443,ONLINE,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
4732,1,1,2016,6,9,14:44,3.28,Chip Transaction,5605026992595823040,Little Neck,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
4733,1,1,2016,6,9,18:52,143.95,Online Transaction,4796006601552743675,ONLINE,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
5359,1,1,2019,5,11,14:05,488.00,Chip Transaction,-7807051024009846392,Rome,...,02/2011,2011,No,3467,0.00,0.00,3343.42,3343.42,33.00,8
5360,1,1,2019,5,11,14:53,14.22,Chip Transaction,9057735476014445185,Rome,...,02/2011,2011,No,3467,0.00,0.00,3343.42,3343.42,33.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6838504,1989,0,2016,11,24,10:13,93.09,Chip Transaction,4859470550592676417,Coshocton,...,02/2011,2011,No,3357279,0.00,0.00,1221.97,1221.97,26.00,5
6838505,1989,0,2016,11,24,10:27,129.23,Online Transaction,-5904116920141006298,ONLINE,...,02/2011,2011,No,3357279,0.00,0.00,1221.97,1221.97,26.00,5
6838506,1989,0,2016,11,24,11:04,303.73,Online Transaction,7035602569409149834,ONLINE,...,02/2011,2011,No,3357279,0.00,0.00,1221.97,1221.97,26.00,5
6838507,1989,0,2016,11,24,11:21,13.71,Online Transaction,-4282466774399734331,ONLINE,...,02/2011,2011,No,3357279,0.00,0.00,1221.97,1221.97,26.00,5


In [25]:
df.loc[(df['Total_Debt'] < 59898.51) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
10957,2,1,2018,1,10,11:24,8.84,Swipe Transaction,2919745757515313025,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
10958,2,1,2018,1,10,11:47,874.49,Swipe Transaction,3189517333335617109,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
10959,2,1,2018,1,10,14:40,2.05,Swipe Transaction,6341249420865687678,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
10960,2,1,2018,1,10,14:53,6.13,Swipe Transaction,6098563624419731578,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
10961,2,1,2018,1,10,16:17,6.04,Swipe Transaction,-727612092139916043,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6854621,1993,1,2019,6,6,15:44,10.16,Chip Transaction,-7146670748125200898,Rome,...,01/2013,2013,No,3365858,265.23,7.00,771.80,771.80,16.00,6
6854622,1993,1,2019,6,7,10:48,106.65,Chip Transaction,6051395022895754231,Rome,...,01/2013,2013,No,3365859,275.39,8.00,781.96,781.96,17.00,6
6854623,1993,1,2019,6,14,15:07,162.00,Chip Transaction,-112121233619748226,Rome,...,01/2013,2013,No,3365860,382.04,9.00,728.03,728.03,16.00,6
6854625,1993,1,2019,6,15,09:02,-162.00,Chip Transaction,-112121233619748226,Rome,...,01/2013,2013,No,3365861,544.04,10.00,890.03,890.03,17.00,6


In [26]:
df.loc[(df['Total_Debt'] >= 59898.51) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
1691,0,2,2016,3,6,11:18,81.93,Chip Transaction,-6680087784759370261,Claremont,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
1692,0,2,2016,3,6,12:10,297.86,Online Transaction,-3220758452254689706,ONLINE,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
2536,0,3,2016,2,23,10:26,244.23,Online Transaction,5763106017265139365,ONLINE,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
2537,0,3,2016,2,23,14:38,22.40,Swipe Transaction,-99077337081867836,Claremont,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
4731,1,1,2016,6,9,10:08,181.74,Online Transaction,-6761066196029841443,ONLINE,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6869755,1997,2,2016,9,4,10:45,1.25,Online Transaction,6455213054093379528,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869757,1997,2,2016,9,4,12:30,116.66,Online Transaction,822647939787511611,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869758,1997,2,2016,9,4,12:40,359.99,Online Transaction,8080934608468946743,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869759,1997,2,2016,9,4,13:45,351.51,Online Transaction,-4282466774399734331,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4


In [27]:
df.loc[(df['Num_Credit_Cards'] >= 3.57) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
1691,0,2,2016,3,6,11:18,81.93,Chip Transaction,-6680087784759370261,Claremont,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
1692,0,2,2016,3,6,12:10,297.86,Online Transaction,-3220758452254689706,ONLINE,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
2536,0,3,2016,2,23,10:26,244.23,Online Transaction,5763106017265139365,ONLINE,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
2537,0,3,2016,2,23,14:38,22.40,Swipe Transaction,-99077337081867836,Claremont,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
4731,1,1,2016,6,9,10:08,181.74,Online Transaction,-6761066196029841443,ONLINE,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6865182,1996,3,2019,6,28,14:38,164.05,Chip Transaction,-8804513634474593447,Rome,...,02/2005,2005,No,3372017,933.53,6.00,1661.63,1661.63,34.00,14
6865183,1996,3,2019,6,28,14:59,9.64,Chip Transaction,-7146670748125200898,Rome,...,02/2005,2005,No,3372017,933.53,6.00,1661.63,1661.63,34.00,14
6865187,1996,3,2019,6,29,14:25,362.00,Chip Transaction,190253443608377572,Rome,...,02/2005,2005,No,3372018,1107.22,8.00,1835.32,1835.32,36.00,14
6865189,1996,3,2019,7,1,11:13,-362.00,Chip Transaction,190253443608377572,Rome,...,02/2005,2005,No,3372020,1469.22,9.00,2217.47,2217.47,40.00,14


In [28]:
df.loc[(df['Num_Credit_Cards'] < 3.57) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
21928,4,0,2016,9,18,13:22,36.68,Chip Transaction,6306878600158076123,San Francisco,...,12/1999,2012,No,12306,0.00,0.00,6796.90,6796.90,73.00,4
25130,5,0,2016,3,31,10:34,11.45,Online Transaction,-3220758452254689706,ONLINE,...,01/2002,2019,No,13483,0.00,0.00,2444.04,2444.04,41.00,-3
25132,5,0,2016,3,31,21:08,471.00,Online Transaction,483490033258680568,ONLINE,...,01/2002,2019,No,13483,0.00,0.00,2444.04,2444.04,41.00,-3
25140,5,0,2016,4,5,15:07,398.93,Online Transaction,483490033258680568,ONLINE,...,01/2002,2019,No,13487,482.45,2.00,2987.96,2987.96,42.00,-3
25141,5,0,2016,4,5,15:16,-471.00,Online Transaction,483490033258680568,ONLINE,...,01/2002,2019,No,13487,482.45,2.00,2987.96,2987.96,42.00,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6869755,1997,2,2016,9,4,10:45,1.25,Online Transaction,6455213054093379528,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869757,1997,2,2016,9,4,12:30,116.66,Online Transaction,822647939787511611,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869758,1997,2,2016,9,4,12:40,359.99,Online Transaction,8080934608468946743,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869759,1997,2,2016,9,4,13:45,351.51,Online Transaction,-4282466774399734331,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4


In [29]:
df.loc[(df['Yearly_Income_-_Person'] < 46707.57) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
10957,2,1,2018,1,10,11:24,8.84,Swipe Transaction,2919745757515313025,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
10958,2,1,2018,1,10,11:47,874.49,Swipe Transaction,3189517333335617109,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
10959,2,1,2018,1,10,14:40,2.05,Swipe Transaction,6341249420865687678,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
10960,2,1,2018,1,10,14:53,6.13,Swipe Transaction,6098563624419731578,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
10961,2,1,2018,1,10,16:17,6.04,Swipe Transaction,-727612092139916043,Rome,...,03/2002,2008,No,6542,0.00,0.00,1608.11,1608.11,49.00,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6869755,1997,2,2016,9,4,10:45,1.25,Online Transaction,6455213054093379528,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869757,1997,2,2016,9,4,12:30,116.66,Online Transaction,822647939787511611,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869758,1997,2,2016,9,4,12:40,359.99,Online Transaction,8080934608468946743,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869759,1997,2,2016,9,4,13:45,351.51,Online Transaction,-4282466774399734331,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4


In [30]:
df.loc[(df['Yearly_Income_-_Person'] >= 46707.57) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
1691,0,2,2016,3,6,11:18,81.93,Chip Transaction,-6680087784759370261,Claremont,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
1692,0,2,2016,3,6,12:10,297.86,Online Transaction,-3220758452254689706,ONLINE,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
2536,0,3,2016,2,23,10:26,244.23,Online Transaction,5763106017265139365,ONLINE,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
2537,0,3,2016,2,23,14:38,22.40,Swipe Transaction,-99077337081867836,Claremont,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
4731,1,1,2016,6,9,10:08,181.74,Online Transaction,-6761066196029841443,ONLINE,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6865182,1996,3,2019,6,28,14:38,164.05,Chip Transaction,-8804513634474593447,Rome,...,02/2005,2005,No,3372017,933.53,6.00,1661.63,1661.63,34.00,14
6865183,1996,3,2019,6,28,14:59,9.64,Chip Transaction,-7146670748125200898,Rome,...,02/2005,2005,No,3372017,933.53,6.00,1661.63,1661.63,34.00,14
6865187,1996,3,2019,6,29,14:25,362.00,Chip Transaction,190253443608377572,Rome,...,02/2005,2005,No,3372018,1107.22,8.00,1835.32,1835.32,36.00,14
6865189,1996,3,2019,7,1,11:13,-362.00,Chip Transaction,190253443608377572,Rome,...,02/2005,2005,No,3372020,1469.22,9.00,2217.47,2217.47,40.00,14


In [31]:
df.loc[(df['Credit_Limit'] < 15423.18) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
2536,0,3,2016,2,23,10:26,244.23,Online Transaction,5763106017265139365,ONLINE,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
2537,0,3,2016,2,23,14:38,22.40,Swipe Transaction,-99077337081867836,Claremont,...,01/2003,2012,No,1865,0.00,0.00,4645.36,4645.36,55.00,4
18013,3,1,2017,11,19,12:06,9.53,Chip Transaction,3017176960763408508,Rome,...,02/2007,2007,No,10016,0.00,0.00,438.10,438.10,12.00,10
18014,3,1,2017,11,19,12:07,510.24,Chip Transaction,-2916542501422915698,Rome,...,02/2007,2007,No,10016,0.00,0.00,438.10,438.10,12.00,10
18015,3,1,2017,11,19,19:30,2.52,Chip Transaction,8181289561763813437,Rome,...,02/2007,2007,No,10016,0.00,0.00,438.10,438.10,12.00,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6869755,1997,2,2016,9,4,10:45,1.25,Online Transaction,6455213054093379528,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869757,1997,2,2016,9,4,12:30,116.66,Online Transaction,822647939787511611,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869758,1997,2,2016,9,4,12:40,359.99,Online Transaction,8080934608468946743,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4
6869759,1997,2,2016,9,4,13:45,351.51,Online Transaction,-4282466774399734331,ONLINE,...,04/2012,2012,No,3374147,0.00,0.00,1070.39,1070.39,20.00,4


In [32]:
df.loc[(df['Credit_Limit'] > 15423.18) & (df['Is_Fraud?'] == 'Yes')]

,User,Card,Year,Month,Day,Time,Amount,Use_Chip,Merchant_Name,Merchant_City,...,Acct_Open_Date,Year_PIN_last_Changed,Card_on_Dark_Web,level_2,rolling_charge_off,rolling_fraud_count,rolling_tran_count,rolling_tran_volume,transaction_count,years_since_pin_change
1691,0,2,2016,3,6,11:18,81.93,Chip Transaction,-6680087784759370261,Claremont,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
1692,0,2,2016,3,6,12:10,297.86,Online Transaction,-3220758452254689706,ONLINE,...,07/2003,2004,No,1272,0.00,0.00,1034.86,1034.86,13.00,12
4731,1,1,2016,6,9,10:08,181.74,Online Transaction,-6761066196029841443,ONLINE,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
4732,1,1,2016,6,9,14:44,3.28,Chip Transaction,5605026992595823040,Little Neck,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
4733,1,1,2016,6,9,18:52,143.95,Online Transaction,4796006601552743675,ONLINE,...,02/2011,2011,No,3051,0.00,0.00,2329.22,2329.22,13.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6854623,1993,1,2019,6,14,15:07,162.00,Chip Transaction,-112121233619748226,Rome,...,01/2013,2013,No,3365860,382.04,9.00,728.03,728.03,16.00,6
6854625,1993,1,2019,6,15,09:02,-162.00,Chip Transaction,-112121233619748226,Rome,...,01/2013,2013,No,3365861,544.04,10.00,890.03,890.03,17.00,6
6854626,1993,1,2019,6,15,13:17,82.78,Chip Transaction,7488618388847074803,Rome,...,01/2013,2013,No,3365861,544.04,10.00,890.03,890.03,17.00,6
6861147,1996,0,2016,10,16,10:11,57.26,Swipe Transaction,-5393221949511984686,Ft Mitchell,...,10/2014,2014,No,3369442,0.00,0.00,590.86,590.86,20.00,2


In [ ]:
zscores = (df - df.mean()) / df.std()
anomalies = df[np.abs(zscores) > 3]

In [ ]:
#anomalies = df[grubbs(df).pvalue < 0.05]

In [ ]:
anomalies = df[mahalanobis(df, df.mean()) > 3]

In [ ]:
lof = LocalOutlierFactor(n_neighbors=20)
lof_scores = lof.fit_predict(df)
anomalies = df[lof_scores < -1]

In [ ]:
df.groupby('Card').agg({'Card': 'sum'})